In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ------------------------------------------------------------------
# config
# ------------------------------------------------------------------

TRAIN_BATCH_SIZE = 32
TRIAN_TXT= '/mogu/lingfei/project/cate4/train_data/cate4_look_merchandise_weibo_train.txt'
NUM_CLASSES =24
INPUT_HEIGHT,INPUT_WIDTH =224, 224
MODEL_FILE = '/mogu/lingfei/project/cate4/resnet50/model_file/' + 'resnet50.json'
BEST_WEIGHTS_FILE = '/mogu/lingfei/project/cate4/resnet50/model_file/'+'weight-{val_acc}.hdf5'
TF_LOG_DIR = '/mogu/lingfei/project/cate4/resnet50/log'

#the epoch of diff steps
EPOCHS1 = 2
EPOCHS2 = 4
EPOCHS3 = 5

#use diff learning rate in diff steps
LR1 = 0.001
LR2 = 0.0005
LR3 = 0.0001

# ------------------------------------------------------------------
# prepare data and data augmentation
# ------------------------------------------------------------------
import cv2
import numpy as np
import os
from PIL import Image
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import random

def load_img_array(image_name, grayscale=False, target_size=(INPUT_HEIGHT, INPUT_WIDTH)):
    img = image.load_img(image_name, grayscale)
    resize_img = img.resize(target_size)
    img_array = image.img_to_array(resize_img)/ 255.0
    return img_array

def random_hue_saturation_value(image,
                                hue_shift_limit=(-15, 15),
                                sat_shift_limit=(-0.025, 0.025),
                                val_shift_limit=(-25, 25),
                                u=0.25):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    return image

def random_transform(image,
                     seed=1,
                     rotation_range=40,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     horizontal_flip=True,
                     u=0.25):
    if np.random.random() < u:
        data_gen_args = dict(rotation_range=rotation_range,
                             width_shift_range=width_shift_range,
                             height_shift_range=height_shift_range,
                             horizontal_flip=horizontal_flip)
        image_data_gen = ImageDataGenerator(**data_gen_args)
        image = image_data_gen.random_transform(image, seed)
    return image

def data_augmentation(image, seed=1):
    image = random_hue_saturation_value(image)
    image = random_transform(image, seed=1)
    return image


#打乱训练样本，然后按照顺序读取batch训练样本。。。。
def train_data_generator(images, batch_size, augment=True, seed=1):
    random.shuffle(images) #打乱训练样本
    while True:
        for base in range(0, len(images), batch_size):
            image_batch = []
            y_batch = []
            for offset in range(batch_size):
                idx = base + offset
                if idx >= len(images):
                    break
                image_name = images[idx].split(' ')[0]
                #print(image_name)
                image_array = load_img_array(image_name, grayscale=False)
                #数据增强开关
                if augment:
                    image_array = data_augmentation(image_array, seed + idx)

                image_batch.append(image_array)
                y_batch.append(images[idx].split(' ')[1])
            image_batch = np.array(image_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            y_batch = to_categorical(y_batch, 24) # 需要onehot编码，生成最后softmax的格式
            yield image_batch, y_batch

        
def get_train_imgs(TRIAN_TXT):
    all_images = []
    with open(TRIAN_TXT,'r') as f:
        for line in f:
            all_images.append(line.strip('\n'))
    f.close()
    random.shuffle(all_images)
    return all_images

# ------------------------------------------------------------------
# create and train model and save model
# ------------------------------------------------------------------

from keras.applications.resnet50 import ResNet50
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


#准备训练数据，切分训练集与测试集
all_train_images = get_train_imgs((TRIAN_TXT))
train_images, validation_images = train_test_split(all_train_images, train_size=0.94, test_size=0.06, random_state=42)
print("Number of train_images: {}".format(len(train_images)))
print("Number of validation_images: {}".format(len(validation_images)))
train_gen = train_data_generator(train_images, TRAIN_BATCH_SIZE, augment=True)
validation_gen = train_data_generator(validation_images, TRAIN_BATCH_SIZE, augment=False)
print("train and valid generator is ok")

steps_per_epoch = len(train_images) / TRAIN_BATCH_SIZE
validation_steps = len(validation_images) / TRAIN_BATCH_SIZE
print("steps_per_epoch:", steps_per_epoch)
print("validation_steps:", validation_steps) 


#回调函数
callbacks = [EarlyStopping(monitor='val_acc',
                           patience=5,
                           verbose=1,
                           min_delta=0.00001,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_acc',
                               factor=0.1,
                               patience=5,
                               verbose=1,
                               epsilon=0.00001,
                               cooldown=0,
                               mode='max'),
             ModelCheckpoint(monitor='val_acc',
                             filepath=BEST_WEIGHTS_FILE,
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1,
                             mode='max'),
             TensorBoard(log_dir=TF_LOG_DIR)]
 

# create the base pre-trained model
#model.load_weights('')
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(INPUT_HEIGHT, INPUT_WIDTH, 3))
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
#x = Dense(128, activation='relu')(x)
# add a logistic layer
print(x.shape)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()

# #-------------------------------------------------------------------------step1
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = True
    
# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer=RMSprop(lr=LR1), loss='categorical_crossentropy', metrics=['accuracy'])
# #save mode
# def save_model(model, model_file):
#     model_json_string = model.to_json()
#     with open(model_file, 'w') as mf:
#         mf.write(model_json_string)
# save_model(model, MODEL_FILE)
# # train the model on the new data for a few epochs
# model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS1,
#                     callbacks=callbacks,
#                     validation_data=validation_gen, validation_steps=validation_steps)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from model. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

#-------------------------------------------------------------------------step2
# we chose to train the some blocks, i.e. we will freeze the rest blocks
# for layer in model.layers[:40]:
#     layer.trainable = False
# for layer in model.layers[40:]:
#     layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.load_weights('/mogu/lingfei/project/cate4/resnet50/model_file/weight-0.09931645290726561.hdf5')
model.compile(optimizer=RMSprop(lr=LR2), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 blocks
# alongside the top Dense layers
model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS3,
                    callbacks=callbacks,
                    validation_data=validation_gen, validation_steps=validation_steps)

#------------------------------------------------------------------------- step3
# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# for layer in model.layers:
#     layer.trainable = True
# model.compile(optimizer=SGD(lr=LR3, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# # we train our model again (this time fine-tuning the top 2 blocks
# # alongside the top Dense layers
# model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS3,
#                     callbacks=callbacks,
#                     validation_data=validation_gen, validation_steps=validation_steps,
#                     workers=3, use_multiprocessing=True)

Number of train_images: 852611
Number of validation_images: 54422
train and valid generator is ok
steps_per_epoch: 26644.09375
validation_steps: 1700.6875
(?, 2048)
Epoch 1/5
26644/26644 [============================>.] - ETA: 0s - loss: 0.7930 - acc: 0.7576
Epoch 00001: val_acc improved from -inf to 0.70051, saving model to /mogu/lingfei/project/cate4/resnet50/model_file/weight-0.7005071478402407.hdf5
26645/26644 [==============================] - 24774s 930ms/step - loss: 0.7930 - acc: 0.7575 - val_loss: 1.0683 - val_acc: 0.7005
Epoch 2/5
26644/26644 [============================>.] - ETA: 0s - loss: 0.6331 - acc: 0.8063
Epoch 00002: val_acc improved from 0.70051 to 0.80958, saving model to /mogu/lingfei/project/cate4/resnet50/model_file/weight-0.8095806842799014.hdf5
26645/26644 [==============================] - 24002s 901ms/step - loss: 0.6331 - acc: 0.8063 - val_loss: 0.6315 - val_acc: 0.8096
Epoch 3/5
26644/26644 [============================>.] - ETA: 0s - loss: 0.5854 - acc: 0

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ------------------------------------------------------------------
# config
# ------------------------------------------------------------------

TRAIN_BATCH_SIZE = 32
TRIAN_TXT= '/mogu/lingfei/project/cate4/train_data/cate4_look_merchandise_weibo_train.txt'
NUM_CLASSES =24
INPUT_HEIGHT,INPUT_WIDTH =224, 224
MODEL_FILE = '/mogu/lingfei/project/cate4/resnet50/model_file/' + 'resnet50.json'
BEST_WEIGHTS_FILE = '/mogu/lingfei/project/cate4/resnet50/model_file/'+'weight-{val_acc}.hdf5'
TF_LOG_DIR = '/mogu/lingfei/project/cate4/resnet50/log'

#the epoch of diff steps
EPOCHS1 = 2
EPOCHS2 = 4
EPOCHS3 = 50

#use diff learning rate in diff steps
LR1 = 0.001
LR2 = 0.0005
LR3 = 0.0005

# ------------------------------------------------------------------
# prepare data and data augmentation
# ------------------------------------------------------------------
import cv2
import numpy as np
import os
from PIL import Image
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import random

def load_img_array(image_name, grayscale=False, target_size=(INPUT_HEIGHT, INPUT_WIDTH)):
    img = image.load_img(image_name, grayscale)
    resize_img = img.resize(target_size)
    img_array = image.img_to_array(resize_img)/ 255.0
    return img_array

def random_hue_saturation_value(image,
                                hue_shift_limit=(-15, 15),
                                sat_shift_limit=(-0.025, 0.025),
                                val_shift_limit=(-25, 25),
                                u=0.25):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    return image

def random_transform(image,
                     seed=1,
                     rotation_range=40,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     horizontal_flip=True,
                     u=0.25):
    if np.random.random() < u:
        data_gen_args = dict(rotation_range=rotation_range,
                             width_shift_range=width_shift_range,
                             height_shift_range=height_shift_range,
                             horizontal_flip=horizontal_flip)
        image_data_gen = ImageDataGenerator(**data_gen_args)
        image = image_data_gen.random_transform(image, seed)
    return image

def data_augmentation(image, seed=1):
    image = random_hue_saturation_value(image)
    image = random_transform(image, seed=1)
    return image


#打乱训练样本，然后按照顺序读取batch训练样本。。。。
def train_data_generator(images, batch_size, augment=True, seed=1):
    random.shuffle(images) #打乱训练样本
    while True:
        for base in range(0, len(images), batch_size):
            image_batch = []
            y_batch = []
            for offset in range(batch_size):
                idx = base + offset
                if idx >= len(images):
                    break
                image_name = images[idx].split(' ')[0]
                #print(image_name)
                image_array = load_img_array(image_name, grayscale=False)
                #数据增强开关
                if augment:
                    image_array = data_augmentation(image_array, seed + idx)

                image_batch.append(image_array)
                y_batch.append(images[idx].split(' ')[1])
            image_batch = np.array(image_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            y_batch = to_categorical(y_batch, 24) # 需要onehot编码，生成最后softmax的格式
            yield image_batch, y_batch

        
def get_train_imgs(TRIAN_TXT):
    all_images = []
    with open(TRIAN_TXT,'r') as f:
        for line in f:
            all_images.append(line.strip('\n'))
    f.close()
    random.shuffle(all_images)
    return all_images

# ------------------------------------------------------------------
# create and train model and save mode
# ------------------------------------------------------------------
from keras.applications.resnet50 import ResNet50
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


#准备训练数据，切分训练集与测试集
all_train_images = get_train_imgs((TRIAN_TXT))
train_images, validation_images = train_test_split(all_train_images, train_size=0.94, test_size=0.06, random_state=42)
print("Number of train_images: {}".format(len(train_images)))
print("Number of validation_images: {}".format(len(validation_images)))
train_gen = train_data_generator(train_images, TRAIN_BATCH_SIZE, augment=True)
validation_gen = train_data_generator(validation_images, TRAIN_BATCH_SIZE, augment=False)
print("train and valid generator is ok")

steps_per_epoch = len(train_images) / TRAIN_BATCH_SIZE
validation_steps = len(validation_images) / TRAIN_BATCH_SIZE
print("steps_per_epoch:", steps_per_epoch)
print("validation_steps:", validation_steps) 


#回调函数
callbacks = [EarlyStopping(monitor='val_acc',
                           patience=5,
                           verbose=1,
                           min_delta=0.00001,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_acc',
                               factor=0.1,
                               patience=5,
                               verbose=1,
                               epsilon=0.00001,
                               cooldown=0,
                               mode='max'),
             ModelCheckpoint(monitor='val_acc',
                             filepath=BEST_WEIGHTS_FILE,
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1,
                             mode='max'),
             TensorBoard(log_dir=TF_LOG_DIR)]
 

# create the base pre-trained model
#model.load_weights('')
base_model = ResNet50(include_top=False, input_shape=(INPUT_HEIGHT, INPUT_WIDTH, 3))
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
#x = Dense(128, activation='relu')(x)
# add a logistic layer
print(x.shape)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()

# #-------------------------------------------------------------------------step1
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
    
# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer=RMSprop(lr=LR1), loss='categorical_crossentropy', metrics=['accuracy'])
# #save mode
# def save_model(model, model_file):
#     model_json_string = model.to_json()
#     with open(model_file, 'w') as mf:
#         mf.write(model_json_string)
# save_model(model, MODEL_FILE)
# # train the model on the new data for a few epochs
# model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS1,
#                     callbacks=callbacks,
#                     validation_data=validation_gen, validation_steps=validation_steps)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from model. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

#-------------------------------------------------------------------------step2
# we chose to train the some blocks, i.e. we will freeze the rest blocks
# for layer in model.layers[:40]:
#     layer.trainable = False
# for layer in model.layers[40:]:
#     layer.trainable = True
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.load_weights('/mogu/lingfei/project/cate4/resnet50/model_file/weight-0.8287089779912244.hdf5')
for layer in base_model.layers:
    layer.trainable = True
model.compile(optimizer=RMSprop(lr=LR3), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 blocks
# alongside the top Dense layers
model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS3,
                    callbacks=callbacks,
                    validation_data=validation_gen, validation_steps=validation_steps)

#------------------------------------------------------------------------- step3
# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# for layer in model.layers:
#     layer.trainable = True
# model.compile(optimizer=SGD(lr=LR3, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# # we train our model again (this time fine-tuning the top 2 blocks
# # alongside the top Dense layers
# model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS3,
#                     callbacks=callbacks,
#                     validation_data=validation_gen, validation_steps=validation_steps,
#                     workers=3, use_multiprocessing=True)

/mogu/lingfei/install/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Number of train_images: 852611
Number of validation_images: 54422
train and valid generator is ok
steps_per_epoch: 26644.09375
validation_steps: 1700.6875
(?, 2048)
Epoch 1/50
26644/26644 [============================>.] - ETA: 0s - loss: 0.5074 - acc: 0.8449
Epoch 00001: val_acc improved from -inf to 0.83336, saving model to /mogu/lingfei/project/cate4/resnet50/model_file/weight-0.8333578332335897.hdf5
26645/26644 [==============================] - 24401s 916ms/step - loss: 0.5074 - acc: 0.8449 - val_loss: 0.5876 - val_acc: 0.8334
Epoch 2/50
26644/26644 [============================>.] - ETA: 0s - loss: 0.5060 - acc: 0.8455
Epoch 00002: val_acc improved from 0.83336 to 0.85528, saving model to /mogu/lingfei/project/cate4/resnet50/model_file/weight-0.8552791150571896.hdf5
26645/26644 [==============================] - 24293s 912ms/step - loss: 0.5060 - acc: 0.8455 - val_loss: 0.4716 - val_acc: 0.8553
Epoch 3/50
22432/26644 [========================>.....] - ETA: 1:02:38 - loss: 0.5017 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ------------------------------------------------------------------
# config
# ------------------------------------------------------------------

TRAIN_BATCH_SIZE = 64
TRIAN_TXT= '/mogu/lingfei/project/cate4/clean_trian_data/clean_train_resnet50-0.75.txt'
NUM_CLASSES =24
INPUT_HEIGHT,INPUT_WIDTH =224, 224
MODEL_FILE = '/mogu/lingfei/project/cate4/resnet50/model_file/fine/' + 'resnet50.json'
BEST_WEIGHTS_FILE = '/mogu/lingfei/project/cate4/resnet50/model_file/fine/'+'weight-{val_acc}.hdf5'
TF_LOG_DIR = '/mogu/lingfei/project/cate4/resnet50/log'

#the epoch of diff steps
EPOCHS1 = 2
EPOCHS2 = 4
EPOCHS3 = 20

#use diff learning rate in diff steps
LR1 = 0.001
LR2 = 0.0005
LR3 = 0.00001

# ------------------------------------------------------------------
# prepare data and data augmentation
# ------------------------------------------------------------------
import cv2
import numpy as np
import os
from PIL import Image
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import random

def load_img_array(image_name, grayscale=False, target_size=(INPUT_HEIGHT, INPUT_WIDTH)):
    img = image.load_img(image_name, grayscale)
    resize_img = img.resize(target_size)
    img_array = image.img_to_array(resize_img)/ 255.0
    return img_array

def random_hue_saturation_value(image,
                                hue_shift_limit=(-15, 15),
                                sat_shift_limit=(-0.025, 0.025),
                                val_shift_limit=(-25, 25),
                                u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    return image

def random_transform(image,
                     seed=1,
                     rotation_range=50,
                     width_shift_range=0.2,
                     height_shift_range=0.2,
                     horizontal_flip=True,
                     u=0.4):
    if np.random.random() < u:
        data_gen_args = dict(rotation_range=rotation_range,
                             width_shift_range=width_shift_range,
                             height_shift_range=height_shift_range,
                             horizontal_flip=horizontal_flip)
        image_data_gen = ImageDataGenerator(**data_gen_args)
        image = image_data_gen.random_transform(image, seed)
    return image

def data_augmentation(image, seed=1):
    image = random_hue_saturation_value(image)
    image = random_transform(image, seed=1)
    return image


#打乱训练样本，然后按照顺序读取batch训练样本。。。。
def train_data_generator(images, batch_size, augment=True, seed=1):
    random.shuffle(images) #打乱训练样本
    while True:
        for base in range(0, len(images), batch_size):
            image_batch = []
            y_batch = []
            for offset in range(batch_size):
                idx = base + offset
                if idx >= len(images):
                    break
                image_name = images[idx].split(' ')[0]
                #print(image_name)
                image_array = load_img_array(image_name, grayscale=False)
                #数据增强开关
                if augment:
                    image_array = data_augmentation(image_array, seed + idx)

                image_batch.append(image_array)
                y_batch.append(images[idx].split(' ')[1])
            image_batch = np.array(image_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            y_batch = to_categorical(y_batch, 24) # 需要onehot编码，生成最后softmax的格式
            yield image_batch, y_batch

        
def get_train_imgs(TRIAN_TXT):
    all_images = []
    with open(TRIAN_TXT,'r') as f:
        for line in f:
            all_images.append(line.strip('\n'))
    f.close()
    random.shuffle(all_images)
    return all_images

# ------------------------------------------------------------------
# create and train model and save mode
# ------------------------------------------------------------------
from keras.applications.resnet50 import ResNet50
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


#准备训练数据，切分训练集与测试集
all_train_images = get_train_imgs((TRIAN_TXT))
train_images, validation_images = train_test_split(all_train_images, train_size=0.925, test_size=0.075, random_state=42)
print("Number of train_images: {}".format(len(train_images)))
print("Number of validation_images: {}".format(len(validation_images)))
train_gen = train_data_generator(train_images, TRAIN_BATCH_SIZE, augment=True)
validation_gen = train_data_generator(validation_images, TRAIN_BATCH_SIZE, augment=False)
print("train and valid generator is ok")

steps_per_epoch = len(train_images) / TRAIN_BATCH_SIZE
validation_steps = len(validation_images) / TRAIN_BATCH_SIZE
print("steps_per_epoch:", steps_per_epoch)
print("validation_steps:", validation_steps) 


#回调函数
callbacks = [EarlyStopping(monitor='val_acc',
                           patience=5,
                           verbose=1,
                           min_delta=0.00001,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_acc',
                               factor=0.1,
                               patience=5,
                               verbose=1,
                               epsilon=0.00001,
                               cooldown=0,
                               mode='max'),
             ModelCheckpoint(monitor='val_acc',
                             filepath=BEST_WEIGHTS_FILE,
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1,
                             mode='max'),
             TensorBoard(log_dir=TF_LOG_DIR)]
 

# create the base pre-trained model
#model.load_weights('')
base_model = ResNet50(include_top=False, input_shape=(INPUT_HEIGHT, INPUT_WIDTH, 3))
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
#x = Dense(128, activation='relu')(x)
# add a logistic layer
print(x.shape)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()

# #-------------------------------------------------------------------------step1
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
    
# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer=RMSprop(lr=LR1), loss='categorical_crossentropy', metrics=['accuracy'])
# #save mode
# def save_model(model, model_file):
#     model_json_string = model.to_json()
#     with open(model_file, 'w') as mf:
#         mf.write(model_json_string)
# save_model(model, MODEL_FILE)
# # train the model on the new data for a few epochs
# model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS1,
#                     callbacks=callbacks,
#                     validation_data=validation_gen, validation_steps=validation_steps)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from model. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

#-------------------------------------------------------------------------step2
# we chose to train the some blocks, i.e. we will freeze the rest blocks
# for layer in model.layers[:40]:
#     layer.trainable = False
# for layer in model.layers[40:]:
#     layer.trainable = True
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.load_weights('/mogu/lingfei/project/cate4/resnet50/model_file/weight-0.857208481868333.hdf5')
# for layer in model.layers[:10]:
#     layer.trainable = False
# for layer in model.layers[10:]:
#     layer.trainable = True
for layer in model.layers:
    layer.trainable = True
model.compile(optimizer=SGD(lr=LR3, momentum=0.9, decay = 0.000001), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 blocks
# alongside the top Dense layers
model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS3,
                    callbacks=callbacks,
                    validation_data=validation_gen, validation_steps=validation_steps)
#------------------------------------------------------------------------- step3
# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# for layer in model.layers:
#     layer.trainable = True
# model.compile(optimizer=SGD(lr=LR3, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# # we train our model again (this time fine-tuning the top 2 blocks
# # alongside the top Dense layers
# model.fit_generator(generator=train_gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS3,
#                     callbacks=callbacks,
#                     validation_data=validation_gen, validation_steps=validation_steps,
#                     workers=3, use_multiprocessing=True)

Number of train_images: 653980
Number of validation_images: 53026
train and valid generator is ok
steps_per_epoch: 10218.4375
validation_steps: 828.53125
(?, 2048)
Epoch 1/20
  139/10218 [..............................] - ETA: 17:25:09 - loss: 4.6027 - acc: 0.4319